## EDA

In [17]:
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd 
import os 
import numpy as np 


In [18]:
# grab our dataset(s)
os.chdir("/Users/jabras/rent_scrape/data/")
df = pd.read_csv('tokyo.csv', sep = ',')
df_2 = pd.read_csv('tokyo_4.csv', sep = ',')

# append the datasets 
test = pd.concat([df, df_2], axis = 0)



In [4]:
test.head()

,rent_price,address,sqr_m,name_place,rei_price,shikikin,maintenence_price,nearest_eki,apartment_type,year_built,ku_name,floor,house_type,Lat,Lon
0,13万円,東京都中野区中野２,25.1m2,ＪＲ中央線 中野駅 8階建 築1年,-,13万円,10000円,ＪＲ中央線/中野駅 歩5分,2K,築1年,東京都中野区中野２,8階建\n,賃貸マンション,NaN,NaN
1,17万円,東京都中野区新井１,46.07m2,西武新宿線 新井薬師前駅 2階建 築13年,17万円,17万円,3000円,西武新宿線/新井薬師前駅 歩8分,2K,築13年,東京都中野区新井１,\n2階建\n,賃貸アパート,NaN,NaN
2,11万円,東京都目黒区碑文谷２,35.05m2,東急東横線 学芸大学駅 3階建 築30年,11万円,11万円,-,東急東横線/学芸大学駅 歩12分,2K,築30年,東京都目黒区碑文谷２,\n3階建\n,賃貸マンション,NaN,NaN
3,8.7万円,東京都文京区本駒込５,30m2,正太駒込マンション,8.7万円,8.7万円,3000円,ＪＲ山手線/駒込駅 歩6分,2K,築53年,東京都文京区本駒込５,\n12階建\n,賃貸マンション,NaN,NaN
4,7.9万円,東京都中野区中央３,30.62m2,水上アパート,7.9万円,7.9万円,3000円,東京メトロ丸ノ内線/新中野駅 歩8分,2K,築37年,東京都中野区中央３,\n2階建\n,賃貸アパート,NaN,NaN


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 449
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rent_price         900 non-null    object 
 1   address            900 non-null    object 
 2   sqr_m              900 non-null    object 
 3   name_place         900 non-null    object 
 4   rei_price          900 non-null    object 
 5   shikikin           900 non-null    object 
 6   maintenence_price  900 non-null    object 
 7   nearest_eki        900 non-null    object 
 8   apartment_type     900 non-null    object 
 9   year_built         900 non-null    object 
 10  ku_name            900 non-null    object 
 11  floor              900 non-null    object 
 12  house_type         900 non-null    object 
 13  Lat                0 non-null      float64
 14  Lon                0 non-null      float64
dtypes: float64(2), object(13)
memory usage: 112.5+ KB


In [61]:
test.floor.unique()

array([ 8,  2,  3, 12,  7,  6, 11, 10,  4,  5,  9, 13,  0, 14, 15, 24, 20])

In [19]:
def convert_yen_to_number(x):
    # remove man and yen sign
    # first check if it's not a nan value
    if x == '-': 
        x = x.replace('-', '0')
        x = float(x)
        return x
    else:
        x = x.replace('万円', '') # how to handle non man yen values 
        x = x.replace('円', '')
        x = float(x)
        return x

In [20]:
# function for year built 
def convert_year_built(x):
    # first check if it's not a nan value
    if x == '-':
        x = x.replace('-', '0')
        x = int(x)
        return x
    elif "新築" in x:
        x = x.replace('新築', '0')
        x = int(x)
        return x
    else:
        x = x.replace('年', '')
        x = x.replace('築', '')
        x = int(x)
        return x

In [21]:
# convert nearest eki into minute_walk and to station name 
def convert_nearest_eki(x):
    # first check if it's not a nan value
    if x == '-' or x == 'nan':
        x = x.replace('-', '0')
        x = x.replace('nan', '0')
        x = int(x)
        return x
    else:
        # split string up to "歩"
        x = x.split('歩')[1]
        # split string up to "分"
        x = x.split('分')[0]
        return int(x)

def eki_name(x):
    # first check if it's not a nan value
    if x == '-' or x == 'nan':
        x = x.replace('-', '0')
        x = x.replace('nan', '0')
        x = str(x)
        return x
    else:
        # split string up to "歩"
        x = x.split('歩')[0]
        return str(x)

In [22]:
def floor_fix(x):
    x = x.replace("\n", "")
    if "地上" in x:
        x = x.split("地上")[1]
        x = x.split('階')[0]
    else:
        x = x.split('階')[0]
    return x

In [23]:
test['maintenence_price'] = test['maintenence_price'].fillna(0)
# replace rei price with 0 if it is nan
test['rei_price'] = test['rei_price'].fillna(0)
# replace shiki price with 0 if it is nan
test['shikikin'] = test['shikikin'].fillna(0)
# drop columns where square meters is nan
test = test.dropna(subset=['sqr_m'])
# replace \n with nothing in all columns 
test = test.replace('\n','', regex=True)
# make square meters a float & drop the m2
test['sqr_m'] = test['sqr_m'].str.split('m').str[0]
test['sqr_m'] = test['sqr_m'].astype(float)
# drop yen sign and make rent price a float
test["rent_price"] = test["rent_price"].apply(convert_yen_to_number)
test["rei_price"] = test["rei_price"].apply(convert_yen_to_number)
test["shikikin"] = test["shikikin"].apply(convert_yen_to_number)
test["maintenence_price"] = test["maintenence_price"].apply(convert_yen_to_number)

# make year built be set from 0 to now 
# fix ku name
test["ku_name"] = test["ku_name"].str.split('東京都').str[1]
test['ku_name'] = test["ku_name"].str.split('区').str[0]

# fix year built
test['year_built'] = test['year_built'].apply(convert_year_built)
# fix floor
test['floor'] = test['floor'].apply(floor_fix)
test['floor'] = test['floor'].fillna(0)
# replace "" with 0
test['floor'] = test['floor'].replace('', '0')
# make floor an int
test['floor'] = test['floor'].astype(int)
# drop lon and lat
test = test.drop(['Lon', 'Lat'], axis = 1)
# convert nearest eki to minute walk and station name
test['eki_walk'] = test['nearest_eki'].apply(convert_nearest_eki)
test['eki_name'] = test['nearest_eki'].apply(eki_name)
# drop nearest eki
test = test.drop(['nearest_eki'], axis = 1)


In [24]:
test.to_csv("tokyo_model.csv", index = False, encoding = 'utf-8')

In [7]:
df = pd.read_csv('tokyo_model.csv', sep = ',')

In [8]:
df.head()

,rent_price,address,sqr_m,name_place,rei_price,shikikin,maintenence_price,apartment_type,year_built,ku_name,floor,house_type,eki_walk,eki_name
0,13.0,東京都中野区中野２,25.10,ＪＲ中央線 中野駅 8階建 築1年,0.0,13.0,10000.0,2K,1,中野,8,賃貸マンション,5,ＪＲ中央線/中野駅
1,17.0,東京都中野区新井１,46.07,西武新宿線 新井薬師前駅 2階建 築13年,17.0,17.0,3000.0,2K,13,中野,2,賃貸アパート,8,西武新宿線/新井薬師前駅
2,11.0,東京都目黒区碑文谷２,35.05,東急東横線 学芸大学駅 3階建 築30年,11.0,11.0,0.0,2K,30,目黒,3,賃貸マンション,12,東急東横線/学芸大学駅
3,8.7,東京都文京区本駒込５,30.00,正太駒込マンション,8.7,8.7,3000.0,2K,53,文京,12,賃貸マンション,6,ＪＲ山手線/駒込駅
4,7.9,東京都中野区中央３,30.62,水上アパート,7.9,7.9,3000.0,2K,37,中野,2,賃貸アパート,8,東京メトロ丸ノ内線/新中野駅


In [12]:
from sklearn.model_selection import train_test_split

df = df.drop(['name_place', 'address', 'eki_name', "apartment_type", "house_type", "ku_name"], axis = 1)
X = df.drop('rent_price', axis = 1)
y = df['rent_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [16]:
print(len(X_train.columns))
print(len(X_test.columns))

7
7


AttributeError: 'Series' object has no attribute 'columns'

In [4]:
df = pd.get_dummies(df, columns = ["apartment_type", "house_type", "ku_name"])


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rent_price               900 non-null    float64
 1   address                  900 non-null    object 
 2   sqr_m                    900 non-null    float64
 3   name_place               900 non-null    object 
 4   rei_price                900 non-null    float64
 5   shikikin                 900 non-null    float64
 6   maintenence_price        900 non-null    float64
 7   year_built               900 non-null    int64  
 8   floor                    900 non-null    int64  
 9   eki_walk                 900 non-null    int64  
 10  eki_name                 900 non-null    object 
 11  apartment_type_1DK       900 non-null    uint8  
 12  apartment_type_1K        900 non-null    uint8  
 13  apartment_type_1LDK      900 non-null    uint8  
 14  apartment_type_2DK       9

In [ ]:
# spatial analysis in python - https://pypi.org/project/pysal/

# https://www.statsmodels.org/stable/examples/notebooks/generated/kernel_density.html
# https://geographicdata.science/book/notebooks/08_point_pattern_analysis.html
# 